Created by: [SmirkyGraphs](https://smirkygraphs.github.io/). Code: [Github](https://github.com/SmirkyGraphs/Python-Notebooks). Source: [US Dept of Treasury](https://home.treasury.gov/policy-issues/coronavirus/assistance-for-state-local-and-tribal-governments/emergency-rental-assistance-program).
<hr>

# States Emergency Rental Assistance Program

During the COVID-19 pandemic 2 programs were enacted to provide rental assistance to help people stay in their homes during the crisis. ERA1 provides up to 25 billion under the Consolidated Appropriations Act, 2021, which was enacted on December 27, 2020. The funds are provided directly to states, U.S. territories, local governments. The currently data does not include any state/local breakdown for ERA2 funding.

This Notebook code is used to get the state grand totals, and the state government totals (no cities/counties) from the excel file of data up to July, notably the data is from prior to the rent moratorium ending.

<hr>

In [1]:
import pandas as pd
import json

In [2]:
remove = ['American Samoa', 'Guam', 'Puerto Rico', 'Northern Mariana Islands', 'Virgin Islands']

with open('./data/files/states.json', 'r') as f:
    states = json.load(f)['states']
    names = list(states.keys())
    names = [x for x in states if x not in remove]

In [3]:
df = pd.read_excel('./data/raw/July-ERA-data.xlsx', sheet_name='ERA1 State & Local Reporting', skipfooter=5, skiprows=2)

rename_cols = {
    'Unnamed: 0': 'location',
    'Unnamed: 11': 'total_allocation'
}

# add grand totals
df['total_assistance'] = df[list(df)[1:6]].sum(axis=1)
df['total_households'] = df[list(df)[6:11]].sum(axis=1)

# rename and keep only wanted columns
df = df.rename(columns=rename_cols)
df = df[['location', 'total_assistance', 'total_households', 'total_allocation']]

# get percent of total allocated spent & not spent total
df['pct_spent'] = round(df['total_assistance'] / df['total_allocation'], 4)
df['not_spent'] = df['total_allocation'] - df['total_assistance']

# get state total rows
state_total = df[df['location'].isin(names)].copy()
state_total['type'] = 'state_total'

# get only state government rows (no local county/city)
state_only = df[df['location'].str.contains('^State of|Common')].copy()
state_only['type'] = 'state_only'

In [4]:
# combine datasets, remove prefix, add abbr
df = pd.concat([state_only, state_total]).reset_index()
df['location'] = df['location'].str.replace('State of ', '').str.replace('Commonwealth of ', '')
df['state_abbr'] = df['location'].map(states)

# replace 1 of the dc duplicates
idx = df[df['state_abbr'] == 'DC'].iloc[0].name
df.loc[df.index == idx, 'type'] = 'state_only'

df.to_csv('./data/clean/states_rental_assistance.csv', index=False)